---
# <div align="center"><font color='green'>  </font></div>
# <div align="center"><font color='green'> COSC 2673/2793 | Machine Learning  </font></div>
## <div align="center"> <font color='green'> Assignment 01: **Partial Solution**</font></div>
---

**This is not a complete solution for assignment 01.** The perpose of this notebook is to provide feedback on assignment 1 and it is not intended to be a compete solution

# Reading the Data

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
%matplotlib inline

plt.style.use('bmh')

In [ ]:
data_train = pd.read_csv('./Assignment1/train_data.csv')
data_test = pd.read_csv('./Assignment1/test_data.csv')

data_train.head()

## Exploratary Data Analysis (EDA)

First lets see what variable types are in our training dataset

In [ ]:
data_train.dtypes

There are several categotical variables in the dataset. Lets assign the correct data type to them. Variables `CCS Procedure Code` and `APR Severity of Illness Code` are read as integer values even though they should be categories. Lets fix those as well.

In [ ]:
for i, col in enumerate(data_train.columns):
    if data_train[col].dtypes != np.int64:
        data_train[col] = data_train[col].astype('category')
        data_test[col] = data_test[col].astype('category')

data_train['APRSeverityOfIllnessCode'] = data_train['APRSeverityOfIllnessCode'].astype('category')
data_train['CCSProcedureCode'] = data_train['CCSProcedureCode'].astype('category')

data_test['APRSeverityOfIllnessCode'] = data_test['APRSeverityOfIllnessCode'].astype('category')
data_test['CCSProcedureCode'] = data_test['CCSProcedureCode'].astype('category')
data_train.dtypes

Check if there are missing values.

In [ ]:
data_train.isnull().sum()

In [ ]:
data_test.isnull().sum()

**There are no missing values in the train/test datasets.**


Lets see how the test and train data are distributed. 

In [ ]:
plt.figure(figsize=(20,20))
for i, col in enumerate(data_train.columns):

    if col != 'LengthOfStay' and  col != 'ID':
        plt.subplot(4,5,i+1)
        if data_train[col].dtypes != np.int64:
            plt.bar(data_train[col].value_counts().index.astype(str), 
                    data_train[col].value_counts().values/np.sum(data_train[col].value_counts().values), 
                    alpha=0.3, color='r')
            plt.bar(data_test[col].value_counts().index.astype(str), 
                    data_test[col].value_counts().values/np.sum(data_test[col].value_counts().values), 
                    alpha=0.3, color='b')
        else:
            plt.hist(data_train[col], alpha=0.3, color='r', density=True)
            plt.hist(data_test[col], alpha=0.3, color='b', density=True)

        plt.title(col)
        plt.xticks(rotation='vertical')

In [ ]:
plt.hist(data_train['LengthOfStay'], alpha=0.3, color='r', density=True)
plt.xticks(rotation='vertical')
plt.show()

**Observations:**
1. Some new `Health Service Area` elements present in the training set. Therefore this column cannot be used in the prediction model. **This information is important when deciding on the test/train split**. 
2. Test set and train set are fairly equally distributed (except for points 1). 
3. Output variable `Length of Stay` column is Highly Skewed. Most data has `Length of Stay` around 2 - Classes will be inbalanced.
4. Sever categorical attributes have high imbalance: `Type of Admission`, `APR Severity of Illness Code`, `Race`, `Emergency Department Indicator`, `Payment Typology 1`
5. `Birth Weight` might have some outliers. Further exploration needed.
6. The value range for numerical attributes vary significantly. Should consider normalization.


Lets investigate column `Birth Weight`

In [ ]:
plt.boxplot(data_train['BirthWeight'])
plt.title('BirthWeight')
plt.show()

print("Number of rows with birth weight > 5000: ", np.sum(data_train['BirthWeight']>5000))

In [ ]:
data_train = data_train[data_train['BirthWeight']<5000]

**Observation:** Several outliers seen in the birthweight column. Can remove the rows that has birthweight larger than 5000. 

Lets now explore the relationship between attributes and the output variable. For this analysis we can use a subset of the data that is balaced. 

In [ ]:
data_train_sample = data_train.groupby('LengthOfStay')
data_train_sample = data_train_sample.apply(lambda x: x.sample(data_train_sample.size().min()*10, replace=True).reset_index(drop=True))

data_train_sample.reset_index(inplace = True, drop=True)
data_train_sample.head()

In [ ]:
data_train_sample['LengthOfStay'].value_counts()

Lets plot the relationship between numerical variables and `Length of Stay`. Scatterplots can be used here, but they are not informative for this dataset. 

In [ ]:
plt.figure(figsize=(30,5))
i=1
for col in data_train_sample.columns:
    if col != 'ID':
        if data_train_sample[col].dtypes == np.int64 and col != 'LengthOfStay':
            plt.subplot(1,10,i)
            sns.boxplot(x='LengthOfStay',y=col,data=data_train_sample)
            i = i+1
            plt.title(col)


plt.xticks(rotation='vertical')
plt.show()

**Observations:** 
1. The median of the birthweight distribution linearly decrease with the length of stay.
2. Some facilities may be specialized in short stay procedures. 

Lets plot the relationship between categorical variables and Length of Stay.

In [ ]:
plt.figure(figsize=(30,5))
i=1
for col in data_train_sample.columns:
  
  if data_train_sample[col].dtypes != np.int64 and col != 'LengthOfStay':
    plt.subplot(1,8,i)
    ax = sns.boxplot(y='LengthOfStay',x=col,data=data_train_sample)
    ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
    i = i+1
    plt.title(col)


plt.xticks(rotation='vertical')
plt.show()

**Observations:**
1. `Type of Admission`,`ARP Severity of Illness Code`,`Emergency Department Indicator` seems to be highly correlated with `Length of Stay`.
2.  Medicare patients seems to be always short stay. 
3. `CCS Procedure Code` also seem to be informative in predicting `Length of Stay`.

Correlation analysis between numberical attributes

In [ ]:
import seaborn as sns

f, ax = plt.subplots(figsize=(11, 9))
corr = data_train_sample.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=90,
    horizontalalignment='right'
);

**Observations**
1. `Length of Stay` show a strong negative corealtion with `Birth Weight` and some positive correlation with `charge in facility`.
2. Strong correlation between `Average cost in county`, `Average charges in county`, `Average cost in facility`, etc... - **Need regularization to overcome these**. 

# Data Preprocessing

Convert categorical variable to one-hot. 

*Should avoid using get_dummies as we have already split test and train.* 

In [ ]:
from sklearn.preprocessing import OneHotEncoder

def oneHotEncodeColumn(df, colName, drop_col, enc_onehot=None):
    if enc_onehot is None:
        enc_onehot = OneHotEncoder(handle_unknown='ignore')
        enc_onehot.fit(df[[colName]])

    onehot_ = enc_onehot.transform(df[[colName]]).toarray()

    for i in range(len(enc_onehot.categories_[0])):
        if enc_onehot.categories_[0][i] != drop_col:
            if type(enc_onehot.categories_[0][i]) == str:
                df[colName + '_' + enc_onehot.categories_[0][i]] = onehot_[:,i]
            else:
                df[colName + '_' + str(enc_onehot.categories_[0][i])] = onehot_[:,i]

    df = df.drop([colName], axis=1)

    return df, enc_onehot

In [ ]:
data_train, gender_enc = oneHotEncodeColumn(data_train, 'Gender', 'U')
data_train, race_enc = oneHotEncodeColumn(data_train, 'Race', 'Other Race')
data_train, ToA_enc = oneHotEncodeColumn(data_train, 'TypeOfAdmission', 'Newborn')
data_train, CCSPC_enc = oneHotEncodeColumn(data_train, 'CCSProcedureCode', -1)
data_train, paymentTop_enc = oneHotEncodeColumn(data_train, 'PaymentTypology', 'Miscellaneous/Other')
data_train, Egcy_enc = oneHotEncodeColumn(data_train, 'EmergencyDepartmentIndicator', 'N')

`APR Severity of Illness Code` is categorical, however the order of its value is important therefore it should be left as numerical. 

In [ ]:
data_train['APRSeverityOfIllnessCode'] = data_train['APRSeverityOfIllnessCode'].astype(np.int64)/4.0

## Normalize Data

Functions to do min-max and robust scaling. You can also do a power transform of some of the attributes, However I have not demostrated that here. 

In [ ]:
from sklearn import preprocessing

def minMaxScale(df, colName, min_max_scaler=None):
    if min_max_scaler is None: 
        min_max_scaler = preprocessing.MinMaxScaler()
        min_max_scaler.fit(df[[colName]])

    df[colName] = min_max_scaler.transform(df[[colName]])
    return df, min_max_scaler


def robustScale(df, colName, min_max_scaler=None):
    if min_max_scaler is None: 
        min_max_scaler = preprocessing.RobustScaler()
        min_max_scaler.fit(df[[colName]])

    df[colName] = min_max_scaler.transform(df[[colName]])
    return df, min_max_scaler

In [ ]:
data_train.describe()

In [ ]:
import seaborn as sns

f, ax = plt.subplots(figsize=(11, 9))
corr = data_train.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=90,
    horizontalalignment='right'
);

# Developing the model

First step is to decide the performance measure: For this We will use **macro f1_score** as:
 - The target class in the data set is heavyly biased to one class.
 - We would like to see similer performance in both classes. (Not high performace in one class and poor performance in other class)
 
Give the complexity of the problem we would like to achive around .65 f1_score. **This needs to be justified**


Next lets split the data. Here I am splitting the data randomly. However a better approch would be use the `HealthServiceArea` to split the data

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split

# data_trainSplit, data_valSplit = train_test_split(data_train, test_size=0.2)
# data_trainSplit, data_testSplit = train_test_split(data_trainSplit, test_size=0.25)

In [ ]:
data_trainSplit = data_train[(data_train['HealthServiceArea'] == 'New York City') 
                             | (data_train['HealthServiceArea'] == 'Southern Tier') ]
data_valSplit = data_train[(data_train['HealthServiceArea'] == 'Long Island') 
                           | (data_train['HealthServiceArea'] == 'Central NY') ]
data_testSplit = data_train[(data_train['HealthServiceArea'] == 'Capital/Adirond') 
                            | (data_train['HealthServiceArea'] == 'Western NY') ]

Minmax scale all numerical attributes except `Birth Weight`. `Birth Weight` had outliers we can use robust scaler (more for demostration).

In [ ]:
cols2scale = ['AverageCostInCounty','AverageChargesInCounty',
              'AverageCostInFacility','AverageChargesInFacility',
              'AverageIncomeInZipCode', 'BirthWeight']

minmax_scaler_hold = dict()

for col in cols2scale:
    data_trainSplit, minmax_scaler_hold[col] =  minMaxScale(data_trainSplit, col)
    data_valSplit, _ =  minMaxScale(data_valSplit, col, minmax_scaler_hold[col])
    data_testSplit, _ =  minMaxScale(data_testSplit, col, minmax_scaler_hold[col])

    
# data_train, minmax_scaler_hold['BirthWeight'] =  robustScale(data_train, 'BirthWeight')
# data_valSplit, _ =  robustScale(data_valSplit, 'BirthWeight', minmax_scaler_hold['BirthWeight'])
# data_testSplit, _ =  robustScale(data_testSplit, 'BirthWeight', minmax_scaler_hold['BirthWeight'])

In [ ]:
X_val = data_valSplit.drop(['LengthOfStay', 'HealthServiceArea', 'ID'], axis=1) #remove data leakage features
y_val = (data_valSplit[['LengthOfStay']]>3).astype(np.int64)

X_train = data_trainSplit.drop(['LengthOfStay', 'HealthServiceArea', 'ID'], axis=1) #remove data leakage features
y_train = (data_trainSplit[['LengthOfStay']]>3).astype(np.int64)

X_test = data_testSplit.drop(['LengthOfStay', 'HealthServiceArea', 'ID'], axis=1) #remove data leakage features
y_test = (data_testSplit[['LengthOfStay']]>3).astype(np.int64)

In [ ]:
print("Train data size: ", y_train.shape[0])
print("Val data size: ", y_val.shape[0])
print("Test data size: ", y_test.shape[0])

In [ ]:
plt.barh(y_train['LengthOfStay'].value_counts().index.astype(str), 
         y_train['LengthOfStay'].value_counts().values, alpha=0.3, color='r')
plt.xlabel('Number of data Instances')
plt.ylabel('Class')
plt.show()

**Observations:** Class 0 has significatly less data instances than class 1. May need balancing.

Lets compute a class balance weights

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', np.unique(y_train), np.ravel(y_train.to_numpy()))
class_weights = dict(zip(np.unique(y_train), class_weights))

plt.barh([str(i) for i in class_weights], [class_weights[i] for i in class_weights], alpha=0.3, color='r')
plt.xlabel('Class Weight')
plt.ylabel('Class')
plt.show()

Functions for fitting model and drawing feature importance

In [ ]:
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score, recall_score
from sklearn.metrics import classification_report

def get_sample_weights(y, class_weights):
    sample_weight = np.ravel(y)*class_weights[1] + (1 - np.ravel(y))*class_weights[0]
    return sample_weight


def fit_classification_model(clf, xtrain, ytrain, xval, yval, class_weights={0: 1, 1: 1}, sample_weights=None, print_report=False):
    clf.fit(xtrain, np.ravel(ytrain), sample_weight=sample_weights)
    ytrain_pred = clf.predict(xtrain)
    yval_pred = clf.predict(xval)
    
    f1_val = f1_score(yval, yval_pred, average='macro')
    f1_train = f1_score(ytrain, ytrain_pred, average='macro')

    # rec_val = recall_score(yval, yval_pred, average='macro')
    # rec_train = recall_score(ytrain, ytrain_pred, average='macro')
    
    bce_val = log_loss(yval, yval_pred, sample_weight=get_sample_weights(yval, class_weights))
    bce_train = log_loss(ytrain, ytrain_pred, sample_weight=get_sample_weights(ytrain, class_weights))
    
    if print_report:
        print(classification_report(yval, yval_pred))
        
    return clf, f1_train, f1_val, bce_train, bce_val



def test_classification_model(clf, xval, yval, print_report=False):
    yval_pred = clf.predict(xval)
    
    rec_val = recall_score(yval, yval_pred, average='macro')
    
    if print_report:
        print(classification_report(yval, yval_pred))
        print(rec_val)
        
    return clf, rec_val, bce_val

In [ ]:
Results = dict()

## Base model 1

### Linear classifier
First lets start with a simple linear classifier. This will set the baseline.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import plot_confusion_matrix

clf = LogisticRegression(max_iter=500)

clf, rec_train, rec_val, bce_train, bce_val = fit_classification_model(clf, X_train, y_train, X_val, y_val, class_weights=class_weights, print_report=True)

print(f'Train set F1-Score: {rec_train:.4f} BCE: {bce_train:.4f}')
print(f'Valid set F1-Score: {rec_val:.4f} BCE: {bce_val:.4f}')

Results['baseline'] = {'clf': clf, 'rec_train': rec_train, 'rec_val': rec_val, 'bce_train':bce_train, 'bce_val': bce_val}

plot_confusion_matrix(clf, X_val, y_val, normalize='true')  
plt.show()

**Observation:** confusion matrix shows that most data instances are classified as 1

We noticed that there is a class imbalance in the dataset. Therefore we need to take action to correct it. one way of doing this is to add a balacing weight. Lets try that first.

In [ ]:
clf = LogisticRegression(class_weight='balanced',max_iter=500)

clf, rec_train, rec_val, bce_train, bce_val = fit_classification_model(clf, X_train, y_train, X_val, y_val, class_weights=class_weights, print_report=True)

print(f'Train set F1-Score: {rec_train:.4f} BCE: {bce_train:.4f}')
print(f'Valid set F1-Score: {rec_val:.4f} BCE: {bce_val:.4f}')

Results['balanced_linear'] = {'clf': clf, 'rec_train': rec_train, 'rec_val': rec_val, 'bce_train':bce_train, 'bce_val': bce_val}

plot_confusion_matrix(clf, X_val, y_val, normalize='true')  
plt.show()

**Observation:** The performance improves when we add the balancing term. Now there are more correct classfications in class 0.

We do not observe overfitting when we fit the logistic regression model. However, We observed multi-coleniarity in the attributesin our EDA. Therefore regularization is a good idea. Lets try this. 

Lasso Regression has one hyper parameter.  Lambda. Lets tune it using the validation data.

In [ ]:
Cs = np.logspace(-2,2,num=10)

val_performace = list()
train_performace = list()
for C in Cs:
    clf = LogisticRegression(max_iter=500, solver='liblinear', class_weight='balanced', penalty='l1', C=C)
    clf, rec_train, rec_val, bce_train, bce_val = fit_classification_model(clf, X_train, y_train, X_val, y_val, class_weights=class_weights)
    
    val_performace.append(bce_val)
    train_performace.append(bce_train)
    print(f'Done .. C = {C:.5f} Train F1 = {rec_train:.3f}, Val F1 = {rec_val:.3f}')

In [ ]:
plt.plot(Cs, train_performace, 'ro')
plt.plot(Cs, val_performace, 'b.')
plt.xscale('log')
plt.legend(['Train Performance','Val Performance'])
plt.xlabel('Model Capacity')
plt.ylabel('Weighted Binary Cross-Entropy')
plt.show()

**Observation:** Very slight overfitting, both Train and val performance are similar for every C value in range 1-10000. Lets select C=0.2 as it seems to be the best value - lowest gap and lowest val loss

In [ ]:
clf = LogisticRegression(max_iter=500, solver='liblinear', class_weight='balanced', penalty='l1', C=0.07743)

clf, rec_train, rec_val, bce_train, bce_val = fit_classification_model(clf, X_train, y_train, X_val, y_val, class_weights=class_weights)

Results['balanced_linear_lasso'] = {'clf': clf, 'rec_train': rec_train, 'rec_val': rec_val, 'bce_train':bce_train, 'bce_val': bce_val}

print(f'Train set F1-Score: {rec_train:.4f} BCE: {bce_train:.4f}')
print(f'Valid set F1-Score: {rec_val:.4f} BCE: {bce_val:.4f}')

plot_confusion_matrix(clf, X_val, y_val, normalize='true')  
plt.show()

**Observation** The performance is similar to the `balanced_linear` classifier.However we may prefer the Lasso model as it gives a simpler model. More on this later.

### Nonlinear Classification

The above observation seem to indicate that the model capacity maybe low. We can increase the capacity by going to polynormial features.
This is also supported by the EDA as we so some non linier relationships between output varialble and some attributes 

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly2 = PolynomialFeatures(2)
poly2.fit(X_train)
X_train_poly2 = poly2.transform(X_train)
X_val_poly2 = poly2.transform(X_val)

Lets fit a balanced regularized model to the polynormial features. Hyper parameter in Lasso is tuned with hold-out validation. **Can also tune the poly degree parameter**


In [ ]:
Cs = np.logspace(-2,2,num=10)

val_performace = list()
train_performace = list()
for C in Cs:
    clf = LogisticRegression(max_iter=500, solver='liblinear', class_weight='balanced', penalty='l1', C=C)
    clf, rec_train, rec_val, bce_train, bce_val = fit_classification_model(clf, X_train_poly2, y_train, X_val_poly2, y_val, class_weights=class_weights)
    
    val_performace.append(bce_val)
    train_performace.append(bce_train)
    print(f'Done .. C = {C:.5f} Train F1 = {rec_train:.3f}, Val F1 = {rec_val:.3f}')

In [ ]:
plt.plot(Cs, train_performace, 'ro')
plt.plot(Cs, val_performace, 'b.')
plt.xscale('log')
plt.legend(['Train Performance','Val Performance'])
plt.xlabel('Model Capacity')
plt.ylabel('Weighted Binary Cross-Entropy')
plt.show()

**Observation:** Very slight overfitting, both Train and val performance are similar for every C value in range 1-10000. Lets select C=0.02 as it seems to be the best value - lowest gap and lowest val loss

In [ ]:
clf = LogisticRegression(max_iter=500, solver='liblinear', class_weight='balanced', penalty='l1', C=0.07743)
clf, rec_train, rec_val, bce_train, bce_val = fit_classification_model(clf, X_train_poly2, y_train, X_val_poly2, y_val, class_weights=class_weights)

Results['balanced_poly2_lasso'] = {'clf': clf, 'rec_train': rec_train, 'rec_val': rec_val, 'bce_train':bce_train, 'bce_val': bce_val}

print(f'Train set F1-Score: {rec_train:.4f} BCE: {bce_train:.4f}')
print(f'Valid set F1-Score: {rec_val:.4f} BCE: {bce_val:.4f}')

plot_confusion_matrix(clf, X_val_poly2, y_val, normalize='true')  
plt.show()

When using polynormial features it introduces a second hyper parameter. The polynormial degree. Lets tune that hyper-parameter. Unfortunately most conputers allow us only to go upto degree 3, as they will run out of memory. therefore we will do this next instead of goiing throuhg a loop.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(3)
poly.fit(X_train)
X_train_poly3 = poly.transform(X_train)
X_val_poly3 = poly.transform(X_val)

In [ ]:
Cs = np.logspace(-2,1,num=10)

val_performace = list()
train_performace = list()
for C in Cs:
    clf = LogisticRegression(max_iter=500, solver='liblinear', class_weight='balanced', penalty='l1', C=C)
    clf, rec_train, rec_val, bce_train, bce_val = fit_classification_model(clf, X_train_poly3, y_train, X_val_poly3, y_val, class_weights=class_weights)
    
    val_performace.append(bce_val)
    train_performace.append(bce_train)
    print(f'Done .. C = {C:.5f} Train Recall = {rec_train:.3f}, Val Recall = {rec_val:.3f}')

In [ ]:
plt.plot(Cs, train_performace, 'ro')
plt.plot(Cs, val_performace, 'b.')
plt.xscale('log')
plt.legend(['Train Performance','Val Performance'])
plt.xlabel('Model Capacity')
plt.ylabel('Weighted Binary Cross-Entropy')
plt.show()

overffiring seen. The best C value is around 0.215

In [ ]:
clf = LogisticRegression(max_iter=500, solver='liblinear', class_weight='balanced', penalty='l1', C=0.04642)
clf, rec_train, rec_val, bce_train, bce_val = fit_classification_model(clf, X_train_poly3, y_train, X_val_poly3, y_val, class_weights=class_weights)

Results['balanced_poly3_lasso'] = {'clf': clf, 'rec_train': rec_train, 'rec_val': rec_val, 'bce_train':bce_train, 'bce_val': bce_val}

print(f'Train set Recall-Score: {rec_train:.4f} BCE: {bce_train:.4f}')
print(f'Valid set Recall-Score: {rec_val:.4f} BCE: {bce_val:.4f}')

plot_confusion_matrix(clf, X_val_poly3, y_val, normalize='true')  
plt.show()

## Base Model 2
Lets try an ADABoost classifier. This is not an eleigible classifier and it is okay to not have tested this. The following code can easyly be converted to a randomforest that is eleigible.

Need to tune hyperparameters. I have not done this proparly.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=200)
clf, rec_train, rec_val, bce_train, bce_val = fit_classification_model(clf, X_train, y_train, X_val, y_val, class_weights=class_weights, sample_weights=get_sample_weights(y_train, class_weights))

Results['adaBoost'] = {'clf': clf, 'rec_train': rec_train, 'rec_val': rec_val, 'bce_train':bce_train, 'bce_val': bce_val}

print(f'Train set F1-Score: {rec_train:.4f} BCE: {bce_train:.4f}')
print(f'Valid set F1-Score: {rec_val:.4f} BCE: {bce_val:.4f}')

plot_confusion_matrix(clf, X_val, y_val, normalize='true')  
plt.show()

# Model comparison and ultimate judgment

First lets compare the results for each classifier

In [ ]:
# !pip install PrettyTable
from prettytable import PrettyTable
t = PrettyTable(['Model', 'Train F1', 'Train BCE', 'Val F1', 'Val BCE'])

for i in Results:
    rec_train = Results[i]['rec_train']
    rec_val = Results[i]['rec_val']
    bce_train = Results[i]['bce_train']
    bce_val = Results[i]['bce_val']
    t.add_row([i, f'{rec_train:.3f}',f'{bce_train:.3f}', f'{rec_val:.3f}', f'{bce_val:.3f}'])

print(t)

In [ ]:
clfs_lin = ['baseline','balanced_linear','balanced_linear_lasso']
clfs_poly2 = ['balanced_poly2_lasso']
clfs_poly3 = ['balanced_poly3_lasso']

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15,10))
ax = axes.flatten()
for i, c in enumerate(clfs_lin):
    plot_confusion_matrix(Results[c]['clf'], X_val, y_val, normalize='true', ax=ax[i], cmap='Blues')  
    ax[i].title.set_text(c)

for i, c in enumerate(clfs_poly2):
    plot_confusion_matrix(Results[c]['clf'], X_val_poly2, y_val, normalize='true', ax=ax[i+len(clfs_lin)], cmap='Blues')  
    ax[i+len(clfs_lin)].title.set_text(c)

for i, c in enumerate(clfs_poly3):
    plot_confusion_matrix(Results[c]['clf'], X_val_poly3, y_val, normalize='true', ax=ax[i+len(clfs_lin)+1], cmap='Blues')  
    ax[i+len(clfs_lin)+1].title.set_text(c)
    
for i, c in enumerate(['adaBoost', ]):
    plot_confusion_matrix(Results[c]['clf'], X_val, y_val, normalize='true', ax=ax[i+len(clfs_lin)+2], cmap='Blues')  
    ax[i+len(clfs_lin)+2].title.set_text(c)   

plt.tight_layout()  
plt.show()

Out of the eligible classifiers the baseline is clearly suboptimal. Howerver the other four have similar Recall-Score with `balanced_poly3_lasso` having the best. Since the Recall-scores are close we need to investigate the models to make a judgment.

Since `balanced_linear` and `balanced_linear_lasso` have similar performance which would be better?

To decide lets look at the feature importance for the classifiers.

In [ ]:
from sklearn.inspection import permutation_importance

def plot_permutation_importance(clf, X_val, y_val):
    r = permutation_importance(clf, X_val, y_val,n_repeats=30)
    inx = np.argsort(r.importances_mean)
    
    plt.barh(X_val.columns[inx], r.importances_mean[inx])
    plt.xticks(rotation='vertical')

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plot_permutation_importance(Results['balanced_linear']['clf'], X_val, y_val)
plt.subplot(1,2,2)
plot_permutation_importance(Results['balanced_linear_lasso']['clf'], X_val, y_val)

plt.show()

Still both look very similar. However the regularized classifier totally ignores some feature e.g. `Gender_?`, `Average ? in County`. These were known to be colinier with other feature in the EDA. Therefore out of the two we can select the `balanced_linear_lasso` - given everything is equal we select the least complex model.

Next lets pick one out of the nonlinier classifiers. Again we can look at the performance as well as other aspects of the classifier like feature importnace:

In [ ]:
target_feature_names = ['x'.join(['{}^{}'.format(pair[0],pair[1]) for pair in tuple if pair[1]!=0]) for tuple in [zip(X_train.columns,p) for p in poly2.powers_]]
X_val_poly2_df = pd.DataFrame(X_val_poly2, columns = target_feature_names)

plt.figure(figsize=(20,100))
plt.subplot(1,2,1)
plot_permutation_importance(Results['balanced_poly2_lasso']['clf'], X_val_poly2_df, y_val)

Out of the two non linier classifiers I would select `balanced_poly2_lasso` as it is simpler and has similer performace to `balanced_poly3_lasso`.

### Ultimate Judgment

Give the two models we have narrowed down eirlier, **I would finally select  `balanced_poly2_lasso`**: this is because it has higher performance and agree with some observations we made during EDA: some attributes had non linier relationship with the target variable

# Analysis of the final model

Here we ask questions about the model we selected and see if it agrees with our view of the problem and our values.

## Biases
How does the model perfrom for different health service areas?

In [ ]:
X_test_poly2 = poly2.transform(X_test)

In [ ]:
_, f1_test, _ = test_classification_model(Results['balanced_poly2_lasso']['clf'], X_test_poly2, y_test)
print('Test set F1 Score: ', f1_test)

In [ ]:
plot_confusion_matrix(Results['balanced_poly2_lasso']['clf'],
                      X_test_poly2, y_test, normalize='true', cmap='Blues')

In [ ]:
rechold = list()
for hsa in data_testSplit['HealthServiceArea'].unique():
    X_test_hsa = X_test_poly2[data_testSplit['HealthServiceArea'] == hsa]
    y_test_hsa = y_test[data_testSplit['HealthServiceArea'] == hsa]
    _, rec_test, _ = test_classification_model(Results['balanced_poly2_lasso']['clf'], X_test_hsa, y_test_hsa)
    rechold.append(rec_test)

plt.barh(data_valSplit['HealthServiceArea'].unique(), rechold)
plt.show()

How does the models perform for different genderes? is there a difference?

In [ ]:
gender = ['Gender_F', 'Gender_M']

rechold = list()
for hsa in gender:
    X_test_hsa = X_test_poly2[data_testSplit[hsa] == 1]
    y_test_hsa = y_test[data_testSplit[hsa] == 1]
    _, rec_test, _ = test_classification_model(Results['balanced_poly2_lasso']['clf'], X_test_hsa, y_test_hsa)
    rechold.append(rec_test)

plt.barh(gender, rechold)
plt.show()

How does the models perform for different races? is there a difference?

In [ ]:
race = ['Race_Black/African American', 'Race_Multi-racial', 'Race_White']

rechold = list()
for hsa in race:
    X_test_hsa = X_test_poly2[data_testSplit[hsa] == 1]
    y_test_hsa = y_test[data_testSplit[hsa] == 1]
    _, rec_test, _ = test_classification_model(Results['balanced_poly2_lasso']['clf'], X_test_hsa, y_test_hsa)
    rechold.append(rec_test)

plt.barh(race, rechold)
plt.show()

# `test_data.csv` Prediction

In [ ]:
data_test = pd.read_csv('./Assignment1/test_data.csv')

for i, col in enumerate(data_test.columns):
    if data_test[col].dtypes != np.int64:
        data_test[col] = data_test[col].astype('category')

data_test['APRSeverityOfIllnessCode'] = data_test['APRSeverityOfIllnessCode'].astype('category')
data_test['CCSProcedureCode'] = data_test['CCSProcedureCode'].astype('category')

In [ ]:
data_test, _ = oneHotEncodeColumn(data_test, 'Gender', 'U', gender_enc)
data_test, _ = oneHotEncodeColumn(data_test, 'Race', 'Other Race', race_enc)
data_test, _ = oneHotEncodeColumn(data_test, 'TypeOfAdmission', 'Newborn', ToA_enc)
data_test, _ = oneHotEncodeColumn(data_test, 'CCSProcedureCode', -1, CCSPC_enc)
data_test, _ = oneHotEncodeColumn(data_test, 'PaymentTypology', 'Miscellaneous/Other', paymentTop_enc)
data_test, _ = oneHotEncodeColumn(data_test, 'EmergencyDepartmentIndicator', 'N', Egcy_enc)

data_test['APRSeverityOfIllnessCode'] = data_test['APRSeverityOfIllnessCode'].astype(np.int64)/4.0

In [ ]:
cols2scale = ['AverageCostInCounty','AverageChargesInCounty',
              'AverageCostInFacility','AverageChargesInFacility',
              'AverageIncomeInZipCode', 'BirthWeight']

for col in cols2scale:
    data_test, _ =  minMaxScale(data_test, col, minmax_scaler_hold[col])

# data_test, _ =  robustScale(data_test, 'BirthWeight', minmax_scaler_hold['BirthWeight'])

In [ ]:
X_test = data_test.drop(['HealthServiceArea', 'ID'], axis=1) #remove data leakage features

In [ ]:
X_test_poly2 = poly2.transform(X_test)

In [ ]:
clf = Results['balanced_poly2_lasso']['clf']
data_test['LengthOfStay'] =clf.predict(X_test_poly2)

In [ ]:
data_test = data_test[['ID','LengthOfStay']]

In [ ]:
data_test.head()

In [ ]:
data_test.to_csv('Ruwan_s123456_predictions.csv', index=False)